In [1]:
from irony import FitsArray, Combine, Calibration, Calculator, Coordinates

In [2]:
fa = FitsArray.from_pattern("[OBS-PATH]/data/*.fit")
fa

FitsArray(@: 140534515429536, nof: 574)

In [3]:
groupped = fa.groupby("IMAGETYP")
groupped

{'Bias Frame': FitsArray(@: 140533891085504, nof: 10),
 'Dark Frame': FitsArray(@: 140533891087136, nof: 30),
 'Flat Field': FitsArray(@: 140533891072064, nof: 24),
 'Light Frame': FitsArray(@: 140533891114528, nof: 510)}

In [4]:
zero_c = Combine(groupped['Bias Frame'])
master_zero = zero_c.zerocombine("median", output="[OBS-PATH]/data/out/master_zero.fit")
master_zero

Fits(@: 140533891001232, file: [OBS-PATH]/data/out/master_zero.fit)

In [5]:
darks = groupped['Dark Frame'].groupby("EXPTIME")
darks

{15.0: FitsArray(@: 140533891421120, nof: 10),
 20.0: FitsArray(@: 140533888343376, nof: 10),
 30.0: FitsArray(@: 140533888340832, nof: 10)}

In [6]:
flats = groupped['Flat Field'].groupby("FILTER")
flats

{'Bessel_B': FitsArray(@: 140533888347264, nof: 8),
 'Bessel_U': FitsArray(@: 140533888336464, nof: 8),
 'Bessel_V': FitsArray(@: 140533888343568, nof: 8)}

In [7]:
data = groupped['Light Frame'].groupby(["EXPTIME", "FILTER"])
data

{(15.0, 'Bessel_V'): FitsArray(@: 140533891420304, nof: 170),
 (20.0, 'Bessel_B'): FitsArray(@: 140533891427840, nof: 170),
 (30.0, 'Bessel_U'): FitsArray(@: 140533887263808, nof: 170)}

In [8]:
calibrated = []
for (exp, fltr), images in data.items():
    dark_c = Combine(darks[exp])
    master_dark = dark_c.darkcombine("median", output=f"[OBS-PATH]/data/out/master_dark_{exp}.fit", override=True)
    
    flat_c = Combine(flats[fltr])
    master_flat = flat_c.flatcombine("median", output=f"[OBS-PATH]/data/out/master_flat_{fltr}.fit", override=True)
    
    calib = Calibration(images)
    calibrated = calib.calibrate(zero=master_zero, dark=master_dark, flat=master_flat, output=f"[OBS-PATH]/data/out/cali/")
    

In [9]:
calibrated.header

,SIMPLE,BITPIX,NAXIS,NAXIS1,NAXIS2,EXTEND,ORIGIN,DATE,IRAF-TLM,OBJECT,...,WAT0_001,WAT1_001,WAT2_001,ZEROCOR,DARKCOR,FLATCOR,CCDSEC,CCDMEAN,CCDMEANT,CCDPROC
image,,,,,,,,,,,,,,,,,,,,,
[OBS-PATH]/data/out/cali/V523_Cas-001U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-08T04:42:41,2022-12-08T04:42:41,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 8 7:42 Zero level correction image is /m...,Dec 8 7:42 Dark count correction image is /m...,Dec 8 7:42 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",57.36434,1354952561,Dec 8 7:42 CCD processing done
[OBS-PATH]/data/out/cali/V523_Cas-002U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-08T04:42:42,2022-12-08T04:42:41,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 8 7:42 Zero level correction image is /m...,Dec 8 7:42 Dark count correction image is /m...,Dec 8 7:42 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",55.54247,1354952562,Dec 8 7:42 CCD processing done
[OBS-PATH]/data/out/cali/V523_Cas-003U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-08T04:42:42,2022-12-08T04:42:42,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 8 7:42 Zero level correction image is /m...,Dec 8 7:42 Dark count correction image is /m...,Dec 8 7:42 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",54.85281,1354952562,Dec 8 7:42 CCD processing done
[OBS-PATH]/data/out/cali/V523_Cas-004U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-08T04:42:42,2022-12-08T04:42:42,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 8 7:42 Zero level correction image is /m...,Dec 8 7:42 Dark count correction image is /m...,Dec 8 7:42 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",54.66858,1354952562,Dec 8 7:42 CCD processing done
[OBS-PATH]/data/out/cali/V523_Cas-005U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-08T04:42:43,2022-12-08T04:42:42,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 8 7:42 Zero level correction image is /m...,Dec 8 7:42 Dark count correction image is /m...,Dec 8 7:42 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",54.87293,1354952563,Dec 8 7:42 CCD processing done
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[OBS-PATH]/data/out/cali/V523_Cas-166U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-08T04:43:40,2022-12-08T04:43:39,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 8 7:43 Zero level correction image is /m...,Dec 8 7:43 Dark count correction image is /m...,Dec 8 7:43 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",1215.21100,1354952620,Dec 8 7:43 CCD processing done
[OBS-PATH]/data/out/cali/V523_Cas-167U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-08T04:43:40,2022-12-08T04:43:40,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 8 7:43 Zero level correction image is /m...,Dec 8 7:43 Dark count correction image is /m...,Dec 8 7:43 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",1586.88500,1354952620,Dec 8 7:43 CCD processing done
[OBS-PATH]/data/out/cali/V523_Cas-168U.fit,True,-32,2,2048,2048,False,NOAO-IRAF FITS Image Kernel July 2003,2022-12-08T04:43:40,2022-12-08T04:43:40,V523_Cas,...,system=physical,wtype=linear,wtype=linear,Dec 8 7:43 Zero level correction image is /m...,Dec 8 7:43 Dark count correction image is /m...,Dec 8 7:43 Flat field image is /mnt/c/Users/...,"[1:2048,1:2048]",2081.61500,1354952620,Dec 8 7:43 CCD processing done
